In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Edalat'

/content/drive/My Drive/Edalat


In [ ]:
#!unzip 'initial.zip'

Archive:  initial.zip
  inflating: table1.csv              
  inflating: table2.csv              
  inflating: table3.csv              


In [ ]:
import pandas as pd

In [ ]:
colnames = ['person_id', 'part_id', 'percent', 'expect']
t1 = pd.read_csv("table1.csv")
t1.columns = colnames
print(t1.shape)
t1.head()

(1484, 4)


,person_id,part_id,percent,expect
0,810,1,0.82,21
1,284,1,0.73,34
2,175,1,0.83,43
3,436,1,0.85,44
4,289,1,0.71,31


In [ ]:
colnames = ['part_id', 'wage']
t2 = pd.read_csv("table2.csv")
t2.columns = colnames
t2.head()

,part_id,wage
0,1,36000
1,2,26000
2,3,20000
3,4,24000
4,5,35000


In [ ]:
colnames = ['person_id', 'part_id', 'month', 'hours']
t3 = pd.read_csv("table3.csv")
t3.columns = colnames
print(t3.shape)
t3.head()

(17808, 4)


,person_id,part_id,month,hours
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14


In [ ]:
df_12 = pd.merge(t1, t2, on='part_id', how='inner')
print(df_12.shape)
df_12.head()

(1484, 5)


,person_id,part_id,percent,expect,wage
0,810,1,0.82,21,36000
1,284,1,0.73,34,36000
2,175,1,0.83,43,36000
3,436,1,0.85,44,36000
4,289,1,0.71,31,36000


In [ ]:
df_123 = pd.merge(df_12, t3, on=['person_id', 'part_id'], how='inner')
print(df_123.shape)
df_123.head()

(17808, 7)


,person_id,part_id,percent,expect,wage,month,hours
0,810,1,0.82,21,36000,1,20
1,810,1,0.82,21,36000,2,13
2,810,1,0.82,21,36000,3,23
3,810,1,0.82,21,36000,4,25
4,810,1,0.82,21,36000,5,14


In [ ]:
df = df_123
df.sort_values(by=['person_id'], inplace=True, ignore_index=True)

In [ ]:
df.loc[df['expect'] > df['hours'], 'lost'] = 1
df.loc[df['expect'] <= df['hours'], 'lost'] = 0
df.head(2)

,person_id,part_id,percent,expect,wage,month,hours,lost
0,1,8,0.86,43,26000,8,38,1
1,1,8,0.86,43,26000,6,42,1


In [ ]:
losts = df.groupby(['person_id'])['lost'].sum() < 6
inds = losts[losts == True].index

df = df[df['person_id'].isin(inds)]
df.head(2)

,person_id,part_id,percent,expect,wage,month,hours,lost
480,26,3,0.64,17,20000,7,20,0
481,26,3,0.64,17,20000,11,22,0


In [ ]:
df_sum = df.groupby(['person_id', 'part_id']).sum(['expect', 'hours'])
df_sum.head()

,,percent,expect,wage,month,hours
person_id,part_id,,,,,
26,3,7.68,204,240000,78,218
63,1,6.60,240,432000,78,261
67,1,7.56,348,432000,78,385
77,9,7.80,384,492000,78,370
128,10,7.92,348,396000,78,353


In [ ]:
ratio = df_sum['hours']/df_sum['expect']
vals = ratio.values
coef = []
for val in vals:
  if (val>1 and val <=1.25):
    c = 0.25
  elif (val>1.25 and val <=1.5):
    c = 0.5
  elif val>1.5:
    c = 1
  else:
    c = 0
  coef.append(c)

In [ ]:
df_sum['coef'] = coef
df_sum

,,percent,expect,wage,month,hours,coef
person_id,part_id,,,,,,
26,3,7.68,204,240000,78,218,0.25
63,1,6.60,240,432000,78,261,0.25
67,1,7.56,348,432000,78,385,0.25
77,9,7.80,384,492000,78,370,0.00
128,10,7.92,348,396000,78,353,0.25
...,...,...,...,...,...,...,...
956,3,8.40,456,240000,78,482,0.25
960,6,8.40,504,408000,78,522,0.25
968,2,8.40,564,312000,78,610,0.25
